In [1]:
import torch 
torch.cuda.current_device()

0

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset, load_metric
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

timit = load_dataset("timit_asr", data_dir='./data')
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

## text normalization    
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

timit = timit.map(remove_special_characters)
show_random_elements(timit["train"].remove_columns(["audio", "file"]))

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/datasets/load.py:922: FutureWarning: The repository for timit_asr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at timit_asr/timit_asr.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


,text
0,probably around midnight give or take an hour either way
1,please sing just the club theme
2,industry interest in safety goes even farther
3,those who are not purists use canned vegetables when making stew
4,a good attitude is unbeatable
5,his fingers fumbled across the bandages
6,they either walked or were driven
7,boy you're stirrin early a sleepy voice said
8,encyclopedias seldom present anecdotal evidence
9,did anyone see my cab


In [5]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

## instantiate an object of the Wav2Vec2CTCTokenizer class.
import json
from transformers import Wav2Vec2CTCTokenizer

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Map:   0%|          | 0/4620 [00:00<?, ? examples/s]

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [6]:
repo_name = "atishayj25/parp-wave2vec"
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/atishayj25/parp-wave2vec/commit/952e7a317daceacd9c47f34f441c803246b0e129', commit_message='Upload tokenizer', commit_description='', oid='952e7a317daceacd9c47f34f441c803246b0e129', pr_url=None, pr_revision=None, pr_num=None)

In [7]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [8]:
import IPython.display as ipd
import numpy as np
import random

def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

# trim out audio sequences that are longer than 4sec. 
max_input_length_in_sec = 4.0
timit["train"] = timit["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Map (num_proc=4):   0%|          | 0/4620 [00:00<?, ? examples/s]

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated

Map (num_proc=4):   0%|          | 0/1680 [00:00<?, ? examples/s]

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated

Filter:   0%|          | 0/4620 [00:00<?, ? examples/s]

In [9]:
from transformers import Wav2Vec2ForCTC

pred_processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")
finetuned_model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo").to("cuda")

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/configuration_utils.py:387: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [10]:
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

/tmp/ipykernel_1336458/3447755518.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/datasets/load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
import torch 

def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = finetuned_model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = pred_processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)

    return batch

# decode Timit test set 
results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

In [12]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

show_random_elements(results)

Test WER: 0.186


,pred_str,text
0,a chosen few will become generals,a chosen few will become generals
1,the toothfarry forgot to come when roger's tooth fell out,the tooth fairy forgot to come when roger's tooth fell out
2,swing your arm as high as you can,swing your arm as high as you can
3,she slipped and spraind her ankle on the steep slow,she slipped and sprained her ankle on the steep slope
4,i'd rather not buy these shoes than be overcharged,i'd rather not buy these shoes than be overcharged
5,too further his presstege he occasionally reads the wall street journal,to further his prestige he occasionally reads the wall street journal
6,their pcrops were two step latters a chair and a polmb fan,their props were two stepladders a chair and a palm fan
7,plan parenthood organizations premote birth control,planned parenthood organizations promote birth control
8,she sounded as though they already existed,she sounded as though they already existed
9,stackd heels are also popular on dressy or teainleargd shoes,stacked heels are also popular on dressy or tailored shoes


In [13]:
finetuned_model.wav2vec2

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [14]:
import torch.nn.utils.prune as prune

def pruning_bert(model, px, model_type='wav2vec_small'):
    """
    prune out wav2vec 2.0 BERT: 12 transformer layers for BASE, and 24 
                                transformer layers for LARGE

    note: position encoding, projection heads, layernorm statistics are not pruned. 
    """
    if model_type == 'wav2vec_small':
        num_transformer_blocks = 12
    elif model_type == 'libri960_big' or model_type == 'xlsr_53_56k':
        num_transformer_blocks = 24
    else:
        print('model type {} not supported'.format(model_type))        
    print('num_transformer_blocks is', num_transformer_blocks)

    parameters_to_prune =[]
    for ii in range(num_transformer_blocks):
        parameters_to_prune.append((model.encoder.layers[ii].attention.k_proj, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.k_proj, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.v_proj, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.v_proj, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.q_proj, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.q_proj, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.out_proj, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.out_proj, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].feed_forward.intermediate_dense, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].feed_forward.intermediate_dense, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].feed_forward.output_dense, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].feed_forward.output_dense, 'bias'))

    parameters_to_prune = tuple(parameters_to_prune)

    # Now globally prune tensors corresponding to parameter_to_prune by applying L1_Unstructued pruning
    prune.global_unstructured(
        parameters_to_prune,        # the paramters list of layers to be pruned
        pruning_method=prune.L1Unstructured,     # prune tensor by removing channels with the lowest L1-norm 
        amount=px,      # amount of tensor pruned, we used 50%
    )
        
def unprune_bert(model, model_type='wav2vec_small'):
    """
    remove pruning forward pre-hook. This is useful when we want to tweek the learned pruned mask, which is used in PARP.
    """
    if model_type == 'wav2vec_small':
        num_transformer_blocks = 12
    elif model_type == 'libri960_big' or model_type == 'xlsr_53_56k':
        num_transformer_blocks = 24
    else:
        print('model type {} not supported'.format(model_type))
    print('num_transformer_blocks is', num_transformer_blocks)

    parameters_to_prune =[]
    for ii in range(num_transformer_blocks):
        parameters_to_prune.append(model.encoder.layers[ii].attention.k_proj)
        parameters_to_prune.append(model.encoder.layers[ii].attention.v_proj)
        parameters_to_prune.append(model.encoder.layers[ii].attention.q_proj)
        parameters_to_prune.append(model.encoder.layers[ii].attention.out_proj)
        parameters_to_prune.append(model.encoder.layers[ii].feed_forward.intermediate_dense)
        parameters_to_prune.append(model.encoder.layers[ii].feed_forward.output_dense)

    for ii in range(0, len(parameters_to_prune)): # applying both weight+bias masks
        prune.remove(parameters_to_prune[ii], 'weight')
        prune.remove(parameters_to_prune[ii], 'bias')

def see_weight_rate(model, model_type='wav2vec_small'):
    """ check a model's zero rate 
    """
    if model_type == 'wav2vec_small':
        num_transformer_blocks = 12
    elif model_type == 'libri960_big' or model_type == 'xlsr_53_56k':
        num_transformer_blocks = 24
    else:
        print('model type {} not supported'.format(model_type))        
    print('num_transformer_blocks is', num_transformer_blocks)

    sum_list_2, zero_sum_2 = 0, 0
    for ii in range(num_transformer_blocks):
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.k_proj.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.k_proj.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.k_proj.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.k_proj.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.v_proj.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.v_proj.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.v_proj.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.v_proj.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.q_proj.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.q_proj.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.q_proj.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.q_proj.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.out_proj.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.out_proj.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.out_proj.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.out_proj.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].feed_forward.intermediate_dense.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].feed_forward.intermediate_dense.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].feed_forward.intermediate_dense.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].feed_forward.intermediate_dense.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].feed_forward.output_dense.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].feed_forward.output_dense.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].feed_forward.output_dense.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].feed_forward.output_dense.bias == 0))

    bert_zero_rate = 100 * zero_sum_2 / sum_list_2
    print('BERT zero rate is {0:.2f}'.format(bert_zero_rate))
    return bert_zero_rate

In [15]:
pruning_rate = 0.5
pruning_bert(finetuned_model.wav2vec2, pruning_rate, model_type='wav2vec_small')
see_weight_rate(finetuned_model.wav2vec2)

num_transformer_blocks is 12
num_transformer_blocks is 12
BERT zero rate is 50.00


50.0

In [16]:
mask_dict = {}; weight_dict = {}
model_dict = finetuned_model.state_dict()

for key in model_dict.keys():
    if 'mask' in key:
        mask_dict[key] = model_dict[key]
    else:
        weight_dict[key] = model_dict[key]

torch.save(mask_dict, 'pruned-w2v2_' + str(pruning_rate) + '_mask.pt')
torch.save(weight_dict, 'pruned-w2v2_' + str(pruning_rate) + '_weight.pt')

In [17]:
# decode Timit test set again
results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

show_random_elements(results)

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

Test WER: 0.928


,pred_str,text
0,hyremerd thw from his hapy pyryid,he remembered them well from his happy period
1,th are yner aodaroryantlwr,the carpet cleaners shampooed our oriental rug
2,hoothorasthylim dyspnsc,who authorized the unlimited expense account
3,thyaidyasnttals yh haus aniso navt,the idea of a central tank with lines to each house is not in itself a novelty
4,ay ayar moany bawory hard,jane may earn more money by working hard
5,don'task my y yaryan oilyrag lai that,don't ask me to carry an oily rag like that
6,donask me ty cyaryanoilyrag lae that,don't ask me to carry an oily rag like that
7,yaosters loaomn yndyasryts,youngsters love common candy as treats
8,tho foarmers laiyfmstyurn mythymns ras nla s,the farmer's life must be arranged to meet the demands of crops and livestock
9,ythots hrgin faverasnt purs,jeff thought you argued in favor of a centrifuge purchase


In [18]:
def apply_pruning_mask(model, mask_dict, model_type='wav2vec_small'):
    """
    apply existing pruning mask to a pre-trained wav2vec 2.0. 
    """
    if model_type == 'wav2vec_small':
        num_transformer_blocks = 12
    elif model_type == 'libri960_big' or model_type == 'xlsr_53_56k':
        num_transformer_blocks = 24
    else:
        print('model type {} not supported'.format(model_type))        
    print('num_transformer_blocks is', num_transformer_blocks)

    parameters_to_prune =[]
    mask_list_w, mask_list_b = [], [] # maks list for weight and bias
    for ii in range(num_transformer_blocks):
        parameters_to_prune.append(model.encoder.layers[ii].attention.k_proj)
        mask_list_w.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.attention.k_proj.weight_mask'])
        mask_list_b.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.attention.k_proj.bias_mask'])
        parameters_to_prune.append(model.encoder.layers[ii].attention.v_proj)
        mask_list_w.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.attention.v_proj.weight_mask'])
        mask_list_b.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.attention.v_proj.bias_mask'])
        parameters_to_prune.append(model.encoder.layers[ii].attention.q_proj)
        mask_list_w.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.attention.q_proj.weight_mask'])
        mask_list_b.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.attention.q_proj.bias_mask'])
        parameters_to_prune.append(model.encoder.layers[ii].attention.out_proj)
        mask_list_w.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.attention.out_proj.weight_mask'])
        mask_list_b.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.attention.out_proj.bias_mask'])
        parameters_to_prune.append(model.encoder.layers[ii].feed_forward.intermediate_dense)
        mask_list_w.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.feed_forward.intermediate_dense.weight_mask'])
        mask_list_b.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.feed_forward.intermediate_dense.bias_mask'])
        parameters_to_prune.append(model.encoder.layers[ii].feed_forward.output_dense)
        mask_list_w.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.feed_forward.output_dense.weight_mask'])
        mask_list_b.append(mask_dict['wav2vec2.encoder.layers.' + str(ii) + '.feed_forward.output_dense.bias_mask'])

    for ii in range(0, len(parameters_to_prune)): # applying both weight+bias masks
        prune.CustomFromMask.apply(parameters_to_prune[ii], 'weight', mask=mask_list_w[ii])
        prune.CustomFromMask.apply(parameters_to_prune[ii], 'bias', mask=mask_list_b[ii])

In [19]:
from transformers import Wav2Vec2ForCTC

# load pre-trained model (not the finetuned one)
pretrained_model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/configuration_utils.py:387: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
pretrained_model = pretrained_model.to('cuda')

In [21]:
# apply the 50% pruning mask back to pre-traiend initialization 
apply_pruning_mask(pretrained_model.wav2vec2, mask_dict)

# double-check the pre-trained model now has 50% sparsity 
see_weight_rate(pretrained_model.wav2vec2)

num_transformer_blocks is 12
num_transformer_blocks is 12
BERT zero rate is 50.00


50.0

In [22]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=32,
  gradient_accumulation_steps=1, 
  evaluation_strategy="steps",
  num_train_epochs=10,           # orignally 30 epochs used, taking ~3 hours
  fp16=False,
  gradient_checkpointing=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=5,
  push_to_hub=True,
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=pretrained_model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)

In [36]:
torch.cuda.empty_cache()

In [37]:
trainer.train()

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Wer
100,5.106100,3.862412,1.000000
200,3.140300,3.308455,1.000000
300,2.979300,3.020615,1.000000


/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argumen

TrainOutput(global_step=320, training_loss=3.692204713821411, metrics={'train_runtime': 2619.5836, 'train_samples_per_second': 15.186, 'train_steps_per_second': 0.122, 'total_flos': 1.0878516087481728e+18, 'train_loss': 3.692204713821411, 'epoch': 10.0})

In [38]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/718M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/atishayj25/parp-wave2vec/commit/c2731b3ec16967d2b14ed2c753d15c9ef526baf1', commit_message='End of training', commit_description='', oid='c2731b3ec16967d2b14ed2c753d15c9ef526baf1', pr_url=None, pr_revision=None, pr_num=None)

In [39]:
import copy
finetuned_pruned_model = copy.deepcopy(finetuned_model)
finetuned_pruned_model.wav2vec2.encoder.layers[11].attention.k_proj.weight_mask

tensor([[1., 1., 1.,  ..., 0., 1., 0.],
        [1., 0., 1.,  ..., 0., 1., 1.],
        [1., 0., 1.,  ..., 0., 0., 1.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 0., 1.]], device='cuda:0')

In [45]:
from safetensors.torch import load_file

#finetuned_pruned_model.load_state_dict(torch.load("atishayj25/parp-wave2vec/checkpoint-300/model.safetensors"))
finetuned_pruned_model.load_state_dict(load_file("atishayj25/parp-wave2vec/checkpoint-300/model.safetensors"))
finetuned_pruned_model.eval()

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [46]:
see_weight_rate(finetuned_pruned_model.wav2vec2)

num_transformer_blocks is 12
BERT zero rate is 50.00


50.0

In [47]:
# decode Timit test set again
def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = finetuned_pruned_model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)
    
    return batch

results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

show_random_elements(results)

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

Test WER: 1.000


,pred_str,text
0,,elderly people are often excluded
1,,i'd ride the subway but i haven't enough change
2,,don't ask me to carry an oily rag like that
3,,i'd ride the subway but i haven't enough change
4,,look sweetheart some fool was
5,,don't ask me to carry an oily rag like that
6,,each untimely income loss coincided with the breakdown of a heating system part
7,,special task forces rescue hostages from kidnappers
8,,her wardrobe consists of only skirts and blouses
9,,iguanas and alligators are tropical reptiles
